In [5]:
from google.colab import drive
drive.mount("/content/gdrive")


#Data Source: https://drive.google.com/drive/folders/1jRNWAAKw6uViCIGe9qot27iLUEJDW0fu?usp=sharing

Mounted at /content/gdrive


In [6]:
import glob
normal = []
for i in glob.glob("/content/gdrive/MyDrive/Neha Singh/Projects/ML/Tuberculosis/Train/Normal/*"):
  normal.append(i)

len(normal)

697

In [7]:
tb = []
for i in glob.glob("/content/gdrive/MyDrive/Neha Singh/Projects/ML/Tuberculosis/Train/Tuberculosis/*"):
  tb.append(i)

len(tb)

695

In [8]:
normal_val = []
for i in glob.glob("/content/gdrive/MyDrive/Neha Singh/Projects/ML/Tuberculosis/Val/Normal/*"):
  normal_val.append(i)

len(normal_val)

7

In [9]:
tb_val = []
for i in glob.glob("/content/gdrive/MyDrive/Neha Singh/Projects/ML/Tuberculosis/Val/Normal/*"):
  tb_val.append(i)

len(tb_val)

7

In [10]:
TRAIN_PATH = "/content/gdrive/MyDrive/Neha Singh/Projects/ML/Tuberculosis/Train"
VAL_PATH = "/content/gdrive/MyDrive/Neha Singh/Projects/ML/Tuberculosis/Val"

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

# CNN

In [12]:
# CNN Based Model in Keras

model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(2,activation='softmax'))

model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 186624)            0

In [14]:
from keras.preprocessing import image

# Train from scratch
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

test_dataset = image.ImageDataGenerator(rescale=1./255)

In [15]:
train_generator = train_datagen.flow_from_directory(
    '/content/gdrive/MyDrive/Neha Singh/Projects/ML/Tuberculosis/Train',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical')

Found 1392 images belonging to 2 classes.


In [16]:
train_generator.class_indices

{'Normal': 0, 'Tuberculosis': 1}

In [17]:
validation_generator = test_dataset.flow_from_directory(
    '/content/gdrive/MyDrive/Neha Singh/Projects/ML/Tuberculosis/Val',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical' )

Found 12 images belonging to 2 classes.


In [18]:
tf.config.run_functions_eagerly(True)

hist = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs = 10,  
    validation_data = validation_generator,
    validation_steps=len(validation_generator)
)

model.save("/content/gdrive/MyDrive/Neha Singh/Projects/ML/Tuberculosis/model_cnn_1.h5")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/10
44/44 [==============================] - 430s 10s/step - loss: 0.7770 - accuracy: 0.6106 - val_loss: 0.3963 - val_accuracy: 1.0000
Epoch 2/10
44/44 [==============================] - 411s 9s/step - loss: 0.2801 - accuracy: 0.8894 - val_loss: 0.1666 - val_accuracy: 1.0000
Epoch 3/10
44/44 [==============================] - 411s 9s/step - loss: 0.3019 - accuracy: 0.8750 - val_loss: 0.3976 - val_accuracy: 0.8333
Epoch 4/10
44/44 [==============================] - 411s 9s/step - loss: 0.2453 - accuracy: 0.9009 - val_loss: 0.2597 - val_accuracy: 0.8333
Epoch 5/10
44/44 [==============================] - 411s 9s/step - loss: 0.2137 - accuracy: 0.9253 - val_loss: 0.0901 - val_accuracy: 1.0000
Epoch 6/10
44/44 [==============================] - 411s 9s/step - loss: 0.2597 - accuracy: 0.9001 - val_loss: 0.4456 - val_accuracy: 0.8333
Epoch 7/10
44/44 [==============================] - 412s 9s/step - loss: 0.2319 - accuracy: 0.8980 - val_loss: 0.1495 - val_accuracy: 0.9167
Epoch 8/10
4

In [3]:
import tensorflow as tf
model = tf.keras.models.load_model("/content/gdrive/MyDrive/Neha Singh/Projects/ML/Tuberculosis/model_cnn_1.h5")
import glob
test = []
for i in glob.glob("/content/gdrive/MyDrive/Neha Singh/Projects/ML/Tuberculosis/Val/Normal/*"):
  testing = load_img(i, target_size=(224,224))
  testing
  x = img_to_array(testing)
  x = x/255
  x = np.expand_dims(x, axis=0)

  #model.predict_classes(x))
  predict_x=model.predict(x)
  classes_x=np.argmax(predict_x,axis=1)
  print(classes_x)
  print(model.predict(x))#confidence score

OSError: ignored

In [ ]:
import glob
test = []
for i in glob.glob("/content/gdrive/MyDrive/Neha Singh/Projects/ML/Tuberculosis/Val/Tuberculosis/*"):
  testing = load_img(i, target_size=(224,224))
  testing
  x = img_to_array(testing)
  x = x/255
  x = np.expand_dims(x, axis=0)
  predict_x=model.predict(x)
  classes_x=np.argmax(predict_x,axis=1)
  print(classes_x)
  #print(model.predict_classes(x))
  print(model.predict(x))#confidence score

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


[1]
[[3.018823e-13 1.000000e+00]]
[1]
[[2.0149467e-21 1.0000000e+00]]
[1]
[[1.5061675e-05 9.9998498e-01]]
[1]
[[1.2025159e-11 1.0000000e+00]]
[1]
[[0.00175379 0.9982462 ]]


In [ ]:
model

In [ ]:
import sklearn.metrics as metrics

Y_test = [1,1,1,1,1,0,0,0,0,0,0,0]
prediction_cnn = [1,1,1,0,1,0,0,0,0,0,0,0]

print('Confusion Matrix: CNN')
print(metrics.confusion_matrix(Y_test, prediction_cnn))
print('\nClassification Report:')
print(metrics.classification_report(Y_test, prediction_cnn))

Confusion Matrix: CNN
[[7 0]
 [1 4]]

Classification Report:
              precision    recall  f1-score   support

           0       0.88      1.00      0.93         7
           1       1.00      0.80      0.89         5

    accuracy                           0.92        12
   macro avg       0.94      0.90      0.91        12
weighted avg       0.93      0.92      0.91        12

